## Importing all necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import requests # library to handle requests

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Libraries imported.


## Reading the GeoJSON file that contains the neighborhoods of Zurich

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
zurich_data = pd.read_json(body, orient='values')
zurich_data.head()

,type,name,features
0,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [4]:
# Extract features into a dataframe
zurich_data_features = zurich_data['features']
zurich_data_features.head()

0    {'type': 'Feature', 'geometry': {'type': 'Poin...
1    {'type': 'Feature', 'geometry': {'type': 'Poin...
2    {'type': 'Feature', 'geometry': {'type': 'Poin...
3    {'type': 'Feature', 'geometry': {'type': 'Poin...
4    {'type': 'Feature', 'geometry': {'type': 'Poin...
Name: features, dtype: object

In [5]:
# Define the dataframe columns for a neighborhoods dataframe
column_names = ['Borough','Neighborhood', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
# Extract necessary neighborhood information into the dataframe: name, latitude, longitude
for data in zurich_data_features:
    neighborhood_name = data['properties']['name'] 

    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': "Zürich",
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, 
                                           ignore_index=True)
    

In [8]:
neighborhoods.shape

(22, 4)

In [9]:
# How many neighborhoods of Zurich did we get
print('The dataframe has {} neighborhoods.'.format(
        len(neighborhoods['Neighborhood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 22 neighborhoods.


In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Zürich,Hirslanden,47.363057,8.570403
1,Zürich,Witikon,47.359977,8.600887
2,Zürich,Leimbach,47.331480,8.509313
3,Zürich,Wollishofen,47.341491,8.532956
4,Zürich,Enge,47.360931,8.533140
5,Zürich,Riesbach,47.354570,8.556241
6,Zürich,Affoltern,47.423281,8.509221
7,Zürich,Seebach,47.423695,8.542830
8,Zürich,Oerlikon,47.409348,8.545117
9,Zürich,Höngg,47.407904,8.496424


## Visualize the neighborhoods of Zurich on the map

In [11]:
# Get the geographical cooordinates of Zurich
address = 'Zurich, Zurich'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Zurich are 47.3744489, 8.5410422.


In [12]:
# create map of Zurich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

## Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q' # your Foursquare ID
CLIENT_SECRET = 'F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ' # your Foursquare Secret
ACCESS_TOKEN = 'EUXOMD4V042AJ0VBANQVMRQDPSALDTJLWFD4CVMNIMNYIPVE' # Foursquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q
CLIENT_SECRET:F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ


## Exploring Zurich neighborhoods

In [14]:
#Lets take the first neighborhood in the dataframe
neighborhoods.loc[0, 'Neighborhood']

'Hirslanden'

In [15]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hirslanden are 47.3630568387, 8.5704034113.


# Get the top 100 venues that are in Hirslanden within a radius of 500 meters.

In [16]:
# Create the GET request URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q&client_secret=F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ&v=20180605&ll=47.3630568387,8.5704034113&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60734ed5b2cc423f2ef186eb'},
 'response': {'headerLocation': 'Kreis 7',
  'headerFullLocation': 'Kreis 7, Zürich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 47.3675568432, 'lng': 8.577034546600979},
   'sw': {'lat': 47.358556834199995, 'lng': 8.56377227599902}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6b93162d9976b016990c3a',
       'name': 'Marcellino, Pane e Vino',
       'location': {'address': 'Forchstrasse 168',
        'lat': 47.360202,
        'lng': 8.565569,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.360202,
          'lng': 8.565569}],
        'distance': 483,
        'postalCode': '8032',
        'cc': 'CH',
        'city': 'Zür

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,"Marcellino, Pane e Vino",Mediterranean Restaurant,47.360202,8.565569
1,Wilder Mann,Swiss Restaurant,47.360341,8.566796
2,Ristorante Antiquario,Italian Restaurant,47.360676,8.566550
3,Klusplatz,Plaza,47.364025,8.566751
4,Bistro Le Puy,French Restaurant,47.359382,8.568897


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


## Explore neighborhoods in Zurich

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
zurich_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude']
                                )

Hirslanden
Witikon
Leimbach
Wollishofen
Enge
Riesbach
Affoltern
Seebach
Oerlikon
Höngg
Unterstrass
Wipkingen
Schwamendingen
Oberstrass
Altstetten
Aussersihl
Altstadt
Fluntern
Hottingen
Albisrieden
Wiedikon
Industriequartier


In [23]:
# How many venues in the Zurich neighborhoods did Foursquare return
print(zurich_venues.shape)
zurich_venues.head()

(503, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant


In [24]:
# Check unique categories we received for Zurich venues
print('There are {} unique categories.'.format(len(zurich_venues['Venue Category'].unique())))

There are 144 unique categories.


In [25]:
# Which neighborhoods have most venues
zurich_venues.groupby('Neighborhood').count().sort_values(by=['Venue'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Industriequartier,100,100,100,100,100,100
Altstadt,86,86,86,86,86,86
Aussersihl,60,60,60,60,60,60
Oerlikon,43,43,43,43,43,43
Riesbach,41,41,41,41,41,41
Enge,31,31,31,31,31,31
Unterstrass,22,22,22,22,22,22
Wipkingen,19,19,19,19,19,19
Altstetten,18,18,18,18,18,18


In [26]:
zurich_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station
6,Hirslanden,47.363057,8.570403,Coop Pronto,47.363991,8.566771,Grocery Store
7,Hirslanden,47.363057,8.570403,Sonnenberg,47.366290,8.567369,Park
8,Hirslanden,47.363057,8.570403,Kluspark,47.364476,8.564989,Park
9,Hirslanden,47.363057,8.570403,California House,47.365324,8.564940,Hotel


In [27]:
zurich_venues['Venue Category'].unique()

array(['Mediterranean Restaurant', 'Swiss Restaurant',
       'Italian Restaurant', 'Plaza', 'French Restaurant', 'Tram Station',
       'Grocery Store', 'Park', 'Hotel', 'Supermarket', 'Bus Station',
       'Soccer Field', 'Tennis Court', 'Moroccan Restaurant',
       'Restaurant', 'Music Venue', 'Cheese Shop', 'Community Center',
       'Irish Pub', 'Pool', 'Harbor / Marina', 'Gas Station',
       'History Museum', 'Sauna / Steam Room', 'BBQ Joint',
       'Thai Restaurant', 'Burger Joint', 'Cupcake Shop', 'Bar',
       'Train Station', 'Middle Eastern Restaurant', 'Chinese Restaurant',
       'Coffee Shop', 'Japanese Restaurant', 'Beach', 'Movie Theater',
       'Café', 'Garden', 'Taverna', 'Seafood Restaurant', 'Bakery',
       'Art Museum', 'Museum', 'Mexican Restaurant', 'Beer Garden',
       'Wine Bar', 'Playground', 'Sushi Restaurant', 'Spa', 'Bistro',
       'Food Court', 'Department Store', 'Athletics & Sports',
       'Pizza Place', 'Bowling Alley', 'Steakhouse',
       'Vie

In [58]:
# Let's check now many neighborhoods have train stations
neigh_with_train = zurich_venues[zurich_venues['Venue Category'] == 'Train Station' ].reset_index(drop=True)
neigh_with_train

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station
1,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station
2,Oerlikon,47.409348,8.545117,Bahnhof Oerlikon,47.411978,8.544489,Train Station
3,Aussersihl,47.379690,8.521656,Zürich Vorbahnhof,47.382500,8.523395,Train Station


In [63]:
# Narrow the choice of Zurich neighborhoods by considering only those with train stations
subset = zurich_venues.loc[zurich_venues['Neighborhood'].isin(['Enge', 'Affoltern','Oerlikon','Aussersihl'])]
subset

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,Enge,47.360931,8.533140,Seebad Enge,47.361547,8.536754,Pool
31,Enge,47.360931,8.533140,Ristorante Il Tartufo,47.360155,8.533372,Italian Restaurant
32,Enge,47.360931,8.533140,L'Altro,47.360220,8.534218,Italian Restaurant
33,Enge,47.360931,8.533140,Arboretum (Rentenwiese),47.362665,8.536119,Park
34,Enge,47.360931,8.533140,Museum Rietberg,47.358943,8.529728,History Museum
35,Enge,47.360931,8.533140,Sauna am See,47.361763,8.536999,Sauna / Steam Room
36,Enge,47.360931,8.533140,Rieterpark,47.358115,8.530444,Park
37,Enge,47.360931,8.533140,FIFA World Football Museum,47.363655,8.531420,History Museum
38,Enge,47.360931,8.533140,Alden Hotel Splügenschloss,47.364623,8.534344,Hotel
39,Enge,47.360931,8.533140,Hafen Enge Beiz,47.361182,8.535880,BBQ Joint


In [96]:
# We select the neighborhood with the highest number of restaurants and supermarkets
rest_sup=subset[subset['Venue Category'].str.contains("Restaurant")]
rest_sup

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
31,Enge,47.360931,8.533140,Ristorante Il Tartufo,47.360155,8.533372,Italian Restaurant
32,Enge,47.360931,8.533140,L'Altro,47.360220,8.534218,Italian Restaurant
40,Enge,47.360931,8.533140,Restaurant Belvoirpark Zürich,47.357614,8.533223,Restaurant
41,Enge,47.360931,8.533140,Vee's Bistro - Thai Food - Take away,47.364968,8.532719,Thai Restaurant
45,Enge,47.360931,8.533140,Samigo Amusement,47.358889,8.536977,Restaurant
46,Enge,47.360931,8.533140,Ascot Restaurant,47.363676,8.532172,Restaurant
52,Enge,47.360931,8.533140,Sousou,47.363676,8.532308,Middle Eastern Restaurant
58,Enge,47.360931,8.533140,Suan-Long Enge,47.363900,8.531562,Chinese Restaurant
60,Enge,47.360931,8.533140,Fujiya of Japan,47.364301,8.532160,Japanese Restaurant
113,Oerlikon,47.409348,8.545117,Asiaway,47.408093,8.547275,Vietnamese Restaurant


In [93]:
# How many restaurants are in the selected neighborhoods
print(rest_sup.groupby(['Neighborhood']).size())

Neighborhood
Aussersihl    25
Enge           9
Oerlikon      18
dtype: int64


In [97]:
rest_sup=rest_sup.append(subset[subset['Venue Category'].str.contains("Supermarket")])
rest_sup

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
31,Enge,47.360931,8.533140,Ristorante Il Tartufo,47.360155,8.533372,Italian Restaurant
32,Enge,47.360931,8.533140,L'Altro,47.360220,8.534218,Italian Restaurant
40,Enge,47.360931,8.533140,Restaurant Belvoirpark Zürich,47.357614,8.533223,Restaurant
41,Enge,47.360931,8.533140,Vee's Bistro - Thai Food - Take away,47.364968,8.532719,Thai Restaurant
45,Enge,47.360931,8.533140,Samigo Amusement,47.358889,8.536977,Restaurant
46,Enge,47.360931,8.533140,Ascot Restaurant,47.363676,8.532172,Restaurant
52,Enge,47.360931,8.533140,Sousou,47.363676,8.532308,Middle Eastern Restaurant
58,Enge,47.360931,8.533140,Suan-Long Enge,47.363900,8.531562,Chinese Restaurant
60,Enge,47.360931,8.533140,Fujiya of Japan,47.364301,8.532160,Japanese Restaurant
113,Oerlikon,47.409348,8.545117,Asiaway,47.408093,8.547275,Vietnamese Restaurant


In [100]:
# How many restaurants are in the selected neighborhoods with train stations
print(rest_sup.groupby(['Neighborhood']).size())

Neighborhood
Affoltern      1
Aussersihl    25
Enge          11
Oerlikon      21
dtype: int64


In [102]:
# Which neighborhoods have most venues
zurich_venues.groupby('Neighborhood').count().sort_values(by=['Venue'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Industriequartier,100,100,100,100,100,100
Altstadt,86,86,86,86,86,86
Aussersihl,60,60,60,60,60,60
Oerlikon,43,43,43,43,43,43
Riesbach,41,41,41,41,41,41
Enge,31,31,31,31,31,31
Unterstrass,22,22,22,22,22,22
Wipkingen,19,19,19,19,19,19
Altstetten,18,18,18,18,18,18


In [101]:
#res = requests.get(url).json()
#res

In [45]:
# Search for stores nearby
search_query = 'store'
latitude = neigh_with_train.loc[3,'Venue Latitude']
longitude = neigh_with_train.loc[3,'Venue Longitude']
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q&client_secret=F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ&ll=47.382500437679305,8.52339506149292&oauth_token=EUXOMD4V042AJ0VBANQVMRQDPSALDTJLWFD4CVMNIMNYIPVE&v=20180605&query=restaurant&radius=500&limit=100


In [54]:
# assign relevant part of JSON to venues
stores = res['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(stores)
dataframe.shape

(24, 19)

In [33]:
# one hot encoding
zurich_onehot = pd.get_dummies(zurich_venues[['Venue Category']], prefix="", prefix_sep="")
#zurich_onehot = pd.get_dummies(neigh_with_train[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zurich_onehot['Neighborhood'] = zurich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zurich_onehot.columns[-1]] + list(zurich_onehot.columns[:-1])
zurich_onehot = zurich_onehot[fixed_columns]

zurich_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
zurich_onehot.shape

(503, 145)

In [ ]:
zurich_onehot

In [35]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
zurich_grouped = zurich_onehot.groupby('Neighborhood').mean().reset_index()
zurich_grouped

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Affoltern,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,0.00000,0.000000,0.00,0.00000,0.0,0.000000,0.000000,0.000000,0.000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Albisrieden,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000

In [36]:
zurich_grouped.shape

(22, 145)

In [37]:
num_top_venues = 5

for hood in zurich_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = zurich_grouped[zurich_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Affoltern----
                venue  freq
0         Supermarket   0.2
1  Athletics & Sports   0.2
2         Bus Station   0.2
3       Train Station   0.2
4    Department Store   0.2


----Albisrieden----
           venue  freq
0         Bakery   0.2
1    Bus Station   0.2
2   Tram Station   0.2
3  Grocery Store   0.2
4     Playground   0.2


----Altstadt----
              venue  freq
0               Bar  0.08
1              Café  0.08
2  Swiss Restaurant  0.07
3             Plaza  0.06
4             Hotel  0.06


----Altstetten----
         venue  freq
0       Bakery  0.11
1  Supermarket  0.11
2        Hotel  0.11
3          Gym  0.06
4         Food  0.06


----Aussersihl----
                venue  freq
0                 Bar  0.18
1    Swiss Restaurant  0.08
2          Restaurant  0.07
3  Italian Restaurant  0.05
4                Café  0.05


----Enge----
                venue  freq
0                Park  0.16
1          Restaurant  0.10
2                 Bar  0.06
3         Superm